# Домашнее задание 2

Условие: дана таблица с колонками (id, name, salary, managerid). Необходимо написать код на spark, который создает эту таблицу (данные указаны ниже) и в результате выдаст таблицу, в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=13a6838699d47e6561b7fbe96add1036ac69c2f8a14b7916b107f5595148659e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [3]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

## Исходные данные:

In [4]:
data = {'id': [1,2,3,4],
        'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary': ['70', '80', '60', '90'],
        'ManagerId': ['3', '4', 'Null', 'Null']}
df_pd = pd.DataFrame(data)
df_pd

,id,Name,Salary,ManagerId
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


## Создаем DataFrame:

In [5]:
spark = SparkSession.builder.appName('Homework2').getOrCreate()

df = spark.createDataFrame(df_pd)
df.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



## Создаем вспомогательный датафрейм и соединяем:

In [6]:
managers_df = df.select('id', 'Salary').withColumnRenamed('id', 'manager_id').withColumnRenamed('Salary', 'manager_salary')
managers_df.show()

+----------+--------------+
|manager_id|manager_salary|
+----------+--------------+
|         1|            70|
|         2|            80|
|         3|            60|
|         4|            90|
+----------+--------------+



In [7]:
joined = df.join(managers_df, df['ManagerId'] == managers_df['manager_id'], 'inner')
joined.show()

+---+-----+------+---------+----------+--------------+
| id| Name|Salary|ManagerId|manager_id|manager_salary|
+---+-----+------+---------+----------+--------------+
|  1|  Joe|    70|        3|         3|            60|
|  2|Henry|    80|        4|         4|            90|
+---+-----+------+---------+----------+--------------+



## Фильтруем по условию и выводим результат:

In [8]:
joined.filter(joined['Salary'] > joined['manager_salary']).select(joined.Name).show()

+----+
|Name|
+----+
| Joe|
+----+

